# Data wrangle omics studies and Summary_of_primary_data.csv

In [ ]:
import pickle
import random
import pandas as pd
import numpy as np
import statistics as stats
from matplotlib import pyplot as plt
import time
import re

# omics studies

In [ ]:
#p = open("../../data/intermediate_data_02/matches", "rb")
matches = pd.read_pickle("../../data/intermediate_data_02/matches").drop_duplicates()
matches[:5]

In [ ]:
matches.columns = ['transcript_id', 'num_detected', 'source', 'num_sampled']
matches.num_sampled = float(1)
matches['annotation_version']='v5.5'
matches.num_detected = matches.num_detected.apply(lambda x: float(1) if x > 0 else float(0))
#matches.num_sampled = matches.num_sampled.apply(lambda x: float(x))
matches['num_manipulated'] = float(0)
matches[:5]

# primary_data

In [ ]:
primary_data = pd.read_excel("../../data/raw_data/Summary_of_primary_data.xlsx", 'Sheet1')
primary_data = primary_data[['Genus', 'Species','First_author','Publication_year','Type_of_study','Genes_sampled','Genes_manipulated', 'Genes_with_effect',"Genes_no_effect"]]
primary_data[:3]
subset = primary_data[primary_data.Genus=='Chlamydomonas'][primary_data.Species=='reinhardtii']
subset[:3]

In [ ]:
for col in subset.columns[4:]:
    subset[col] = subset[col].apply(lambda x: tuple(x.split("; ")) if isinstance(x,str) else np.NaN) 
subset.head()

In [ ]:
print(subset.columns)

#### Convert `20201019_sampled_genes_key.csv` to dictionary format with `symbol` as keys and `transcript_id` as values

In [ ]:
sampled_genes_key = pd.read_csv("../01_compile_raw_data/20201019_sampled_genes_key.csv")
print(sampled_genes_key.head())
#print()
key = dict(zip(sampled_genes_key.symbol, sampled_genes_key.transcript_id))
print(list(key.items())[:5])

In [ ]:
transformed_subset = pd.DataFrame(columns = ['transcript_id','num_sampled', 'num_manipulated', 'num_detected', 'source'])
for index, row in subset.iterrows():
    source = row.First_author + "_" + str(row.Publication_year)
    for i in range(0,3): 
        if type(row[i+5]) != float:
            for gene in row[i+5]: #Genes_sampled
                output = [0]*(i)+[1]+[0]*(2-i)
                try: 
                    gene = key[re.sub(r" |;","", gene)]
                except: gene = re.sub(r" |;","", gene)
                output = pd.Series([gene]+output+[source], index = transformed_subset.columns)
                transformed_subset = transformed_subset.append(output, ignore_index = True)
                #print(output)

In [ ]:
transformed_subset['annotation_version'] = 'v5.5'
transformed_subset['num_sampled'] = transformed_subset.apply(lambda x: x.num_detected if x.num_detected >0 else x.num_manipulated, axis = 1)

### Merge matches and primary_data to one dataframe

In [ ]:
merged = pd.concat([transformed_subset, matches], ignore_index=True)
grouped = merged.groupby(['transcript_id', 'source']).sum()
print(grouped[:3])

In [ ]:
transcript_list = [list(i)[0] for i in grouped.index]
source_list = [list(i)[1] for i in grouped.index]

df= pd.DataFrame.from_dict({"num_detected": grouped.num_detected,
                            "num_sampled": grouped.num_sampled,
                            'num_manipulated': grouped.num_manipulated}).reset_index()
df.head()

In [ ]:
#Check if any num_detected > num_sampled, because this shouldn't happen
df[df.num_detected>df.num_sampled]

In [ ]:
with open("../../data/intermediate_data_02/sampled_genes.pk", "wb") as f:
    pickle.dump(df, f)